# Small Noise Limit in the Linear Gaussian Setting

In this notebook, we investigate the small noise limit for a special class of inverse problems.
Suppose we intend to infer unknowns $u\in\mathbb{R}^d$ from data $y\in\mathbb{R}^k$. We further
assume the following restrictions on the inverse problem:

1. Linear forward model: $G(u)=Au,\ A\in\mathbb{R}^{k\times d}$
2. Gaussian prior: $u \sim \mathcal{N}(u_{pr}, C_{pr})$
3. Gaussian additive noise: $\eta =\gamma\eta_0,\ \eta_0\sim\mathcal{N}(0,\Gamma_{noise}),\ \gamma\in\mathbb{R}^+$
4. Forward solution and noise are independent: $u \perp \eta$

It follows immediately that the likelihood is given as a normal distribution,
$$ y|u \sim \mathcal{N}(y-Au, \gamma^2\Gamma_{noise}) $$

Furthermore, the posterior density is a product of (independent) Gaussians, meaning that it is
again Gaussian. In particular, we have that
$$ 
\begin{gather*}
\pi_y(u) \sim \mathcal{N}(u_{post}, C_{post}), \\
m_{post} = (A^T\Gamma_{noise}^{-1}A + \gamma^2 C_{pr}^{-1})^{-1}(A^T\Gamma_{noise}^{-1}y + \gamma^2 C_{pr}^{-1}u_{pr}), \\
C_{post} = \gamma^2(A^T\Gamma_{noise}^{-1}A + \gamma^2C_{pr}^{-1})^{-1}
\end{gather*}
$$

In this **linear Gaussian Setting**, we want to explore how the posterior behaves for $\gamma\to 0$ for different
scenarios of the forward model and data.

### General Functions

In [ ]:
# Necessary libraries
import numpy as np
from scipy.stats import multivariate_normal
import ipywidgets as widgets
import matplotlib.pyplot as plt

# Interactive plotting and style
%matplotlib widget
plt.close('all')
plt.style.use('bmh')

# Layout of our slider and textbox widgets
slider_layout = {'style': {'description_width': 'initial'},
                 'layout': widgets.Layout(width='35%'),
                 'continuous_update': True} 

textbox_layout = {'style': {'description_width': 'initial'},
                  'layout': widgets.Layout(width='20%'),
                  'continuous_update': False}

We firstly define some general routines that we need for our calculations. The function `generate_observations`
produces data in the variables in the form of normally distributed random variables. It takes the noise scale $\gamma$,
the underlying true value $m_{true}$ and the forward matrix $A$ as inputs. We define $\Gamma_{noise} = I$ and
further choose $C_{pr} = c_{pr} I$ for some (positive) scalar $c_{pr}$.
The function `compute_posterior_parameters` utilizes the above relations to compute the mean and covariance of the posterior,
given $\gamma, y, m_{pr}, c_{pr}$ and $A$.


**Exercise : Fill out the below function bodies or implement your own versions for the generation of noise and computation
of the posterior under the above assumptions.**

In [ ]:
# 1) Generate data for given noise scale, true parameter and forward matrix
def generate_observations(noise_scale, true_parameter, forward_matrix, seed=123456):

    # observations should be a 1D numpy array
    # Tip: Use the scipy stats module to generate multivariate normal random variables

    return observations

# 2) Compute mean vector and covariance matrix of the Gaussian posterior
def compute_posterior_parameters(noise_scale,
                                 observations,
                                 prior_mean,
                                 prior_cov,
                                 forward_matrix):
    
    # mean_posterior should be 1D array, cov_posterior a 2D array
    
    return mean_posterior, cov_posterior

### Overdetermined Case

We firstly consider the overdetermined case $d < k$. In particular, a scalar unknown $u$ and 2D data $y$. The forward model matrix a is given as $A=\begin{pmatrix} 1 \\ 1 \end{pmatrix}$. Clearly, we have that Null(A)=0.

**Exercise: Compute and visualize the posterior for the overdetermined case and different noise scales $\gamma$.
You can use the code in below cell that uses the functions `generate_observations` and `compute_posterior_parameters`.
What are your observations?**

In [ ]:
# Settings
TRUE_PARAMETER_OVERDETERMINED = 1
FORWARD_MATRIX_OVERDETERMINED = np.array(((1,),(1,)))

# Visualize posterior
def visualize_posterior_overdetermined(noise_scale,
                                       true_parameter,
                                       prior_mean,
                                       prior_cov,
                                       forward_matrix,
                                       mpl_axis):
    prior_mean = np.array((prior_mean,))

    # First function enters here
    observations = generate_observations(noise_scale,
                                         true_parameter,
                                         forward_matrix)
    
    # Second function enters here
    mean, cov = compute_posterior_parameters(noise_scale,
                                             observations,
                                             prior_mean,
                                             prior_cov,
                                             forward_matrix)

    sample_space = np.linspace(-2, 2, 1000, endpoint=True)
    posterior_pdf = multivariate_normal.pdf(sample_space, mean, cov)

    mpl_axis.clear()
    mpl_axis.set_title(rf'Posterior for $\gamma = {noise_scale}$')
    mpl_axis.set_xlabel(r'$u$')
    mpl_axis.set_ylabel(r'$p(u|y)$')
    mpl_axis.set_xlim(np.min(sample_space), np.max(sample_space))
    mpl_axis.set_ylim(0, 1.1*np.max(posterior_pdf))
    mpl_axis.plot(sample_space, posterior_pdf, color='royalblue')
    mpl_axis.axvline(true_parameter, color='black', linestyle='--', alpha=0.5)

# Sliders for parameter variations
slider_noise_scale = widgets.FloatSlider(value=1, min=0.01, max=1, step=0.01,
                                       description='Noise scale:',
                                       **slider_layout)
slider_prior_mean = widgets.FloatSlider(value=0, min=-1, max=1, step=0.05,
                                       description='Prior mean:',
                                       **slider_layout)
slider_prior_cov = widgets.FloatSlider(value=1, min=0.1, max=10, step=0.05,
                                       description='Prior covariance:',
                                       **slider_layout)

# Start interactive visualization
_, ax = plt.subplots()
interactive_plot = widgets.interact(visualize_posterior_overdetermined,
                                    true_parameter=widgets.fixed(TRUE_PARAMETER_OVERDETERMINED),
                                    forward_matrix=widgets.fixed(FORWARD_MATRIX_OVERDETERMINED),
                                    noise_scale=slider_noise_scale,
                                    prior_mean=slider_prior_mean,
                                    prior_cov=slider_prior_cov,
                                    mpl_axis=widgets.fixed(ax))


## Underdetermined case

Secondly, we investigate the undetermined case with d > k. We consider a 2D variable $u$ with model matrix $A=\begin{pmatrix} 1 & 0 \end{pmatrix}$, along with a 1D data point $y$. Note that Null(A) $\neq$ 0, in a way that only the first component of the unknown affects the model output. This implies that the data can not inform the posterior regarding the second component of the unknown.

**Exercise: Compute and visualize the posterior for the underdetermined case and different noise scales $\gamma$.
You can use the code in below cell that uses the functions `generate_observations` and `compute_posterior_parameters`.
What can you observe this time?**

In [ ]:
# Settings
TRUE_PARAMETER_UNDERDETERMINED = np.array((1, 1))
FORWARD_MATRIX_UNDERDETERMINED = np.array(((1, 0),))

# Visualize posterior
def visualize_posterior_underdetermined(noise_scale,
                                        true_parameter,
                                        prior_mean_1,
                                        prior_mean_2,
                                        prior_cov,
                                        forward_matrix,
                                        mpl_axes):
    prior_mean = np.array((prior_mean_1, prior_mean_2,))
    true_parameter = np.array(true_parameter)

    # First function enters here
    observations = generate_observations(noise_scale,
                                         true_parameter,
                                         forward_matrix)
    
    # Second function enters here
    mean, cov = compute_posterior_parameters(noise_scale,
                                             observations,
                                             prior_mean,
                                             prior_cov,
                                             forward_matrix)
    
    sample_space_x = np.linspace(-2, 2, 1000, endpoint=True)
    sample_space_y = np.linspace(-2, 2, 1000, endpoint=True)
    samples_x, samples_y = np.meshgrid(sample_space_x, sample_space_y, indexing='xy')
    posterior_pdf = multivariate_normal.pdf(np.dstack((samples_x, samples_y)), mean, cov)

    marginal_pdf_x = np.mean(posterior_pdf, axis=0)
    marginal_pdf_y = np.mean(posterior_pdf, axis=1)
    
    for ax in (mpl_axes[0, 0], mpl_axes[1, 0], mpl_axes[1, 1]):
        ax.clear()
    mpl_axes[1, 0].contourf(samples_x, samples_y, posterior_pdf, cmap='Blues')
    mpl_axes[1, 0].axvline(true_parameter[0], color='black', linestyle='--', alpha=0.5)
    mpl_axes[1, 0].axhline(true_parameter[1], color='black', linestyle='--', alpha=0.5)
    mpl_axes[1, 0].plot(true_parameter[0], true_parameter[1], marker="o", markersize=10, color='firebrick')
    mpl_axes[1, 0].grid('on', linestyle='--')
    mpl_axes[1, 0].set_xlabel(r'$u_1$')
    mpl_axes[1, 0].set_ylabel(r'$u_2$')
    mpl_axes[1, 0].text(1.1, 1.7, r'$p(u_1, u_2 | y)$')

    mpl_axes[0, 0].plot(sample_space_x, marginal_pdf_x, color='tab:blue')
    mpl_axes[1, 1].plot(marginal_pdf_y, sample_space_y, color='tab:blue')
    mpl_axes[0, 1].axis('off')

    for ax in (mpl_axes[0, 0], mpl_axes[1, 1]):
        ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
    for pos in ['left', 'right', 'top']:
        mpl_axes[0, 0].spines[pos].set_visible(False)
    for pos in ['bottom', 'right', 'top']:
        mpl_axes[1, 1].spines[pos].set_visible(False)

# Sliders for parameter variations
slider_noise_scale = widgets.FloatSlider(value=1, min=0.01, max=1, step=0.01,
                                       description='Noise scale:',
                                       **slider_layout)
slider_prior_mean_1 = widgets.FloatSlider(value=0, min=-1, max=1, step=0.05,
                                          description='Prior mean 1:',
                                          **slider_layout)
slider_prior_mean_2 = widgets.FloatSlider(value=0, min=-1, max=1, step=0.05,
                                          description='Prior mean 2:',
                                          **slider_layout)
slider_prior_cov = widgets.FloatSlider(value=1, min=0.1, max=10, step=0.05,
                                       description='Prior covariance:',
                                       **slider_layout)

# Start interactive visualization
plt.style.use('default')
fig, axs = plt.subplots(2, 2, figsize=(7, 7),
                        gridspec_kw={'width_ratios': [3, 1], 
                                     'height_ratios': (1, 3),
                                     'wspace': 0.1,
                                     'hspace': 0.1})

interactive_plot = widgets.interact(visualize_posterior_underdetermined,
                                    true_parameter=widgets.fixed(TRUE_PARAMETER_UNDERDETERMINED),
                                    forward_matrix=widgets.fixed(FORWARD_MATRIX_UNDERDETERMINED),
                                    noise_scale=slider_noise_scale,
                                    prior_mean_1=slider_prior_mean_1,
                                    prior_mean_2=slider_prior_mean_2,
                                    prior_cov=slider_prior_cov,
                                    mpl_axes=widgets.fixed(axs))